In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
#from sklearn.metrics import 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

-----
### Train, valid split

In [2]:
X_train, X_valid, y_train, y_valid = train_test_split(np.load('data/pars_smp_train.npy'), np.load('data/y_smp_train.npy'), test_size=0.2, shuffle=False, random_state=178)

In [ ]:
class MacroEconomicModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MacroEconomicModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Используйте только последний временной шаг для предсказания
        return out

# Определение размеров входных и выходных данных
input_size = 3  # Размерность данных о приросте ВВП, инфляции и процентной ставке
hidden_size = 64  # Размер скрытого состояния RNN
output_size = 15  # Размер параметров модели
num_layers = 2  # Количество слоев LSTM

# Создание экземпляра модели
model = MacroEconomicModel(input_size, hidden_size, output_size, num_layers)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Преобразование данных в тензоры PyTorch
y_smp_train_T = torch.Tensor(y_smp_train)
pars_smp_train_T = torch.Tensor(pars_smp_train)

batch_size = 15
train_dataset = TensorDataset(y_smp_train_T, pars_smp_train_T)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Обучение модели
num_epochs = 10  # Количество эпох обучения
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Проход вперед (forward pass)
        outputs = model(inputs)

        # Вычисление функции потерь
        loss = criterion(outputs, targets)

        # Обратное распространение (backward pass) и оптимизация
        loss.backward()
        optimizer.step()

    # Выводим информацию о процессе обучения
    print(f'Эпоха [{epoch + 1}/{num_epochs}], Потери: {loss.item()}')